In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v23.0 - THE UNDYING GHOST
Envía la TX de Satoshi y SE QUEDA CONECTADO para ver la reacción.
"""

import hashlib
import struct
import socket
import time
import random
import select
import threading
from datetime import datetime

class KaoruBridgeV23:

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_TX = 1

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.fake_txs = []
        self.tx_hashes = {}
        self.print_lock = threading.Lock()

    @staticmethod
    def double_sha256(data):
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def encode_varint(n):
        if n < 0xfd: return bytes([n])
        elif n <= 0xffff: return b'\xfd' + struct.pack('<H', n)
        else: return b'\xfe' + struct.pack('<I', n)

    @staticmethod
    def decode_varint(data, offset=0):
        first = data[offset]
        if first < 0xfd: return first, offset + 1
        elif first == 0xfd: return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        else: return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5

    def log(self, ip, msg, icon=""):
        with self.print_lock:
            t = datetime.now().strftime('%H:%M:%S')
            print(f"   [{t}] {ip:16s} | {icon} {msg}")

    def msg(self, cmd, payload):
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip):
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        # Usamos un User Agent que suene oficial para confundir
        p += b'\x10/Satoshi:26.0.0/'
        p += struct.pack('<i', 870000) # Altura realista
        p += b'\x01'
        return p

    def create_satoshi_tx(self) -> bytes:
        # Genesis Block UTXO TxID
        genesis_txid = bytes.fromhex("4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b")[::-1]

        tx = struct.pack('<I', 1)
        tx += b'\x01' # 1 input
        tx += genesis_txid
        tx += struct.pack('<I', 0)

        # ScriptSig: PUSH "SATOSHI" + Fake Sig
        # Esto es lo que va a fallar la validación, pero el formato es correcto
        message = b"SATOSHI NAKAMOTO - GENESIS MOVE"
        fake_sig = b'\x01' * 70 # Firma falsa no nula para que intente validar

        script_sig = bytes([len(fake_sig)]) + fake_sig + bytes([len(message)]) + message
        tx += self.encode_varint(len(script_sig)) + script_sig
        tx += struct.pack('<I', 0xffffffff)

        # Output
        tx += b'\x01'
        tx += struct.pack('<Q', 50 * 10**8)
        random_hash = hashlib.sha256(str(random.random()).encode()).digest()[:20]
        script_pubkey = b'\x76\xa9\x14' + random_hash + b'\x88\xac'
        tx += self.encode_varint(len(script_pubkey)) + script_pubkey
        tx += struct.pack('<I', 0)

        return tx

    def prepare_payload(self):
        self.fake_txs = []
        self.tx_hashes = {}
        tx = self.create_satoshi_tx()
        tx_hash = self.double_sha256(tx)
        self.fake_txs.append(tx)
        self.tx_hashes[tx_hash] = tx
        print(f"   [SYSTEM] 🧙‍♂️ TX DE SATOSHI LISTA PARA MAINNET")

    def parse_msgs(self, data):
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except: break
        return msgs

    def recv_all(self, sock):
        try:
            sock.setblocking(False)
            ready = select.select([sock], [], [], 0.1)
            if ready[0]:
                return sock.recv(65536)
        except: pass
        return b''

    def get_nodes(self):
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend(ips[:8])
            except: pass
        random.shuffle(nodes)
        return list(set(nodes))

    def attack_node(self, ip: str, duration: int = 300): # 5 minutos de persistencia
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(10)
            self.log(ip, "Conectando...", "🔗")
            sock.connect((ip, 8333))

            sock.send(self.msg('version', self.version_payload(ip)))

            start = time.time()
            trust_phase = True
            tx_sent = False

            genesis = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]

            while time.time() - start < duration:
                data = self.recv_all(sock)
                if not data:
                    time.sleep(0.1)
                    continue

                for cmd, payload in self.parse_msgs(data):

                    if cmd == 'version':
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        self.log(ip, "Handshake. Pidiendo headers...", "🤝")
                        # Pedimos headers para abrir el canal
                        getheaders = struct.pack('<I', 70016) + b'\x01' + genesis + bytes(32)
                        sock.send(self.msg('getheaders', getheaders))

                    elif cmd == 'headers':
                        if trust_phase:
                            trust_phase = False
                            self.log(ip, "¡Canal abierto! INYECTANDO TX SATOSHI...", "💉")
                            time.sleep(0.5)
                            inv = self.encode_varint(len(self.fake_txs))
                            for tx in self.fake_txs:
                                inv += struct.pack('<I', self.MSG_TX)
                                inv += self.double_sha256(tx)
                            sock.send(self.msg('inv', inv))

                    elif cmd == 'ping':
                        # ¡AQUÍ ESTÁ LA CLAVE DE LA PERSISTENCIA!
                        # Respondemos PONG para que no nos desconecten
                        sock.send(self.msg('pong', payload[:8]))
                        self.log(ip, "PING? PONG! (Manteniendo conexión)", "🏓")

                    elif cmd == 'getdata':
                        self.log(ip, "¡¡PIDEN LA TX!! ENVIANDO...", "😱")
                        count, offset = self.decode_varint(payload, 0)
                        for _ in range(count):
                            if offset + 36 > len(payload): break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36
                            if inv_type == self.MSG_TX and inv_hash in self.tx_hashes:
                                sock.send(self.msg('tx', self.tx_hashes[inv_hash]))
                                tx_sent = True
                                self.log(ip, "TX ENVIADA 📦 ESPERANDO REACCIÓN...", "📤")

                    elif cmd == 'reject':
                        # ¡AQUÍ LEEMOS LA RESPUESTA DEL NODO!
                        try:
                            msg_len = payload[0]
                            msg_type = payload[1:1+msg_len].decode()
                            code = payload[1+msg_len]
                            reason_len = payload[2+msg_len]
                            reason = payload[3+msg_len:3+msg_len+reason_len].decode('ascii', errors='ignore')
                            self.log(ip, f"REJECT: {msg_type} -> {reason}", "❌")
                        except:
                            self.log(ip, f"REJECT (Raw hex): {payload.hex()}", "❌")

                        # NO CERRAMOS LA CONEXIÓN. Nos quedamos molestando.
                        self.log(ip, "El nodo nos rechazó, pero seguimos aquí 😎", "👻")

            sock.close()
        except Exception as e:
            pass

    def execute(self, num_threads: int = 15):
        print(f"""
╔══════════════════════════════════════════════════════════════════════════╗
║                      ⚔️  THE UNDYING GHOST v23 ⚔️                        ║
║             Persistencia + Ping/Pong + Lectura de Errores                ║
╚══════════════════════════════════════════════════════════════════════════╝
        """)

        self.prepare_payload()
        nodes = self.get_nodes()
        print(f"   [SYSTEM] 🌐 {len(nodes)} Nodos. Iniciando Ghost Swarm...")
        print("=" * 80)

        threads = []
        for ip in nodes[:num_threads]:
            t = threading.Thread(target=self.attack_node, args=(ip, 300)) # 5 minutos
            t.daemon = True
            t.start()
            threads.append(t)
            time.sleep(0.2)

        # Mantenemos el script vivo
        try:
            while True:
                time.sleep(1)
                if not any(t.is_alive() for t in threads):
                    break
        except KeyboardInterrupt:
            print("\n   [SYSTEM] Deteniendo ataque...")

if __name__ == "__main__":
    bridge = KaoruBridgeV23()
    bridge.execute(num_threads=15)


╔══════════════════════════════════════════════════════════════════════════╗
║                      ⚔️  THE UNDYING GHOST v23 ⚔️                        ║
║             Persistencia + Ping/Pong + Lectura de Errores                ║
╚══════════════════════════════════════════════════════════════════════════╝
        
   [SYSTEM] 🧙‍♂️ TX DE SATOSHI LISTA PARA MAINNET
   [SYSTEM] 🌐 24 Nodos. Iniciando Ghost Swarm...
   [00:20:47] 37.189.168.79    | 🔗 Conectando...
   [00:20:47] 24.50.221.230    | 🔗 Conectando...
   [00:20:47] 37.189.168.79    | 🤝 Handshake. Pidiendo headers...
   [00:20:47] 174.140.231.229  | 🔗 Conectando...
   [00:20:48] 24.50.221.230    | 🤝 Handshake. Pidiendo headers...
   [00:20:48] 37.189.168.79    | 🏓 PING? PONG! (Manteniendo conexión)
   [00:20:48] 161.8.0.102      | 🔗 Conectando...
   [00:20:48] 24.50.221.230    | 🏓 PING? PONG! (Manteniendo conexión)
   [00:20:48] 37.189.168.79    | 💉 ¡Canal abierto! INYECTANDO TX SATOSHI...
   [00:20:48] 24.50.221.230    | 💉 ¡Can